# Advent of Code 2021

## Day 0 : Imports and Utility Functions

In [11]:
def file_to_list(filename, sep="\n") -> list:
    """
    Read an input file and split it using sep as the delimiter.
    """
    with open(filename) as f:
        return f.read().rstrip().split(sep)

## Day 1: Sonar Sweep

### Part 1

Given a sonar report, count the number of times a depth measurement increases from the previous measurement. For example, in the following report, there are 7 measurements that are larger than the previous measurement.

In [2]:
test_report = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]

In [3]:
def day1_part1(report: list[int]):
    n = 0
    for i in reversed(range(1, len(report))):
        if report[i] > report[i-1]:
            n += 1
    return n

day1_part1(test_report)

7

In [4]:
final_report = [int(value) for value in file_to_list("input.txt")]
day1_part1(final_report)

1215

### Part 2

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [6]:
# Number of sliding windows
len(test_report) - 2

8

In [7]:
def day1_part2(report: list[int]):
    n = 0
    for i in reversed(range(3, len(report))):
        if sum(report[i-2:i+1]) > sum(report[i-3:i]):
            n += 1
    return n

day1_part2(test_report)

5

In [8]:
day1_part2(final_report)

1150

## Day 2: Dive!

### Part 1

The submarine has a planned course consisting of a list of commands such as `forward 1`, `down 2`, or `up 3`. The horizontal position and depth both start at `0`. Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [1]:
test_course = ["forward 5", "down 5", "forward 8", "up 3", "down 8", "forward 2"]

In [18]:
def day2_part1(course: list[str]):
    # x: horizontal position, y: depth
    x = y = 0
    for cmd in course:
        d = int(cmd[-1])
        if cmd[0] == "f":
            x += d
        elif cmd[0] == "d":
            y += d
        else:
            y -= d
    return x * y

day2_part1(test_course)

150

In [12]:
final_course = file_to_list("input2.txt")
day2_part1(final_course)

2150351

### Part 2

New interpretation of the commands:

- `down X` increases your aim by X units.
- `up X` decreases your aim by X units.
- `forward X` does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [17]:
def day2_part2(course: list[str]):
    # x: horizontal position, y: depth
    x = y = aim = 0
    for cmd in course:
        d = int(cmd[-1])
        if cmd[0] == "f":
            x += d
            y += aim * d
        elif cmd[0] == "d":
            aim += d
        else:
            aim -= d
    return x * y

day2_part2(test_course)

900

In [16]:
day2_part2(final_course)

1842742223